# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
true = true[:1000]

In [4]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [6]:
corpus = corpus[:100000]

### Генерация исправлений

In [7]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [8]:
from operator import itemgetter
inp_dict = dict(WORDS)
WORDS_sorted= dict(sorted(inp_dict.items(), key=itemgetter(1), reverse=True)) 

In [9]:
def correction(word): 
    candidate = word
    all_deletes_of_word = list(edits1(word).union(known(edits2(word))))
    for i in all_deletes_of_word:
        for key in WORDS_for_delete:
            if i in WORDS_for_delete[key]:
                candidate = key
                return candidate
    return candidate

def edits1(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    return set(w for w in words)

In [10]:
WORDS_for_delete = dict()
for word in WORDS_sorted:
    WORDS_for_delete[word] = list(edits1(word).union(known(edits2(word))))

In [11]:
WORDS_for_delete

{'в': [''],
 'и': [''],
 'на': ['а', '', 'н'],
 'с': [''],
 'по': ['о', 'п', ''],
 'года': ['га', 'гда', 'ода', 'оа', 'год', 'гд', 'го', 'да', 'гоа', 'од'],
 'из': ['и', '', 'з'],
 'году': ['гоу', 'оду', 'гу', 'год', 'го', 'гд', 'оу', 'гду', 'ду', 'од'],
 'от': ['о', '', 'т'],
 'а': [''],
 'не': ['', 'н', 'е'],
 'к': [''],
 'был': ['бы', 'бл', 'л', 'ыл', 'б', 'ы'],
 'для': ['дл', 'л', 'ля', 'д', 'я', 'дя'],
 'костного': ['костого',
  'косног',
  'остого',
  'косого',
  'отного',
  'остноо',
  'осного',
  'костно',
  'кстног',
  'костго',
  'ксного',
  'котного',
  'костноо',
  'кстного',
  'костнго',
  'костног',
  'костог',
  'кстноо',
  'костнг',
  'котого',
  'костоо',
  'косного',
  'котноо',
  'стного',
  'коного',
  'остнго',
  'котнго',
  'кстнго',
  'остного',
  'ктного',
  'кстого',
  'котног',
  'остног',
  'коснго',
  'косноо'],
 'мозга': ['оза',
  'озг',
  'мга',
  'ога',
  'мог',
  'зга',
  'озга',
  'моа',
  'мзг',
  'мога',
  'моз',
  'мза',
  'моза',
  'мзга',
  'мозг']

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [12]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [13]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.36273726273726276
0.11051419800460476
0.599517629493511


Этот алгоритм работает значительно быстрее